### Import Libraries

In [3]:
import cv2
import numpy as np
import os
import time

### Send Whtsp Msg

In [4]:
import pywhatkit 
from os import environ 
# datetime module to get current time
from datetime import datetime

def wpmsg():
    now = datetime.now()            # Get current time
    hr = int(now.strftime("%H"))    # Current Hour
    min = int( now.strftime("%M"))  # Current mint
    number = environ.get('MOBILE')
    pywhatkit.sendwhatmsg(number,"Hi Deepak, Someone tried to access your app", hr, min+1, wait_time=15)

### Send Email

In [5]:
import smtplib
import os
import getpass
import imghdr
from os import environ 
from email.message import EmailMessage

def send_email():
    
    message = EmailMessage()
    
    # All variable values are set inside system's environmental variable  
    senderMail = environ.get('SENDER_MAIL')   # passing sender mail
    senderPass = environ.get('SENDER_PASS')   # passing sender mail passwd
    recvMail = environ.get('RECV_MAIL')       # passing reciever mail
    
    print("Sending mail....")
    
    message['subject'] = "This is text code by deepak"
    message['from'] = senderMail
    message['to'] = recvMail
    
    # It will only display if html content is not visible
    message.set_content("Welcome to Face recognition app" )
    html_message = open("mail.html").read()
    message.add_alternative(html_message , subtype = "html")
    
    with open("./image/Crop.jpg" , "rb") as attach_file:
        image_name = attach_file.name
        image_type = imghdr.what(attach_file.name)
        image_data = attach_file.read()
        
    message.add_attachment(image_data , maintype = "image",
                          subtype = image_type , filename = "FaceLock.jpg"
                          )
    
    # To connect with gmail server
    with smtplib.SMTP_SSL("smtp.gmail.com",465) as smtp:
        smtp.login(senderMail, senderPass)
        smtp.send_message(message)
        
    print("Email Sent sucessfully")

### AWS CLI Access

In [13]:
import subprocess

def aws_cli():
    if face_actual == 70:
        unlock = cv2.imread("./image/unlock.jpg")
        
        cv2.putText(unlock, "Access", (45, 50) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(unlock, "Granted", (45, 100) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow("unlock" , unlock)
        cv2.waitKey(4000)
        cv2.destroyAllWindows()
        
        print("=============================================")
        print("Launching aws ec2 instance....")
        instance = os.system("aws ec2 run-instances  --image-id ami-0ad704c126371a549 --count 1 --instance-type t2.micro --tag-specifications  ResourceType=instance,Tags=[{Key=Name,Value=Task6}]")
        if instance == 0:
                print("EC2 instance launched!!!! \n")
                os.system("chrome https://ap-south-1.console.aws.amazon.com/ec2/v2/home?region=ap-south-1#Instances:instanceState=running")
                time.sleep(2)
        else:
            print("There's something error here! Instance not deployed")      
            
        print("=============================================")
        print("Launching EBS volume....")    
        ebs_vol = os.system("aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone ap-south-1a  --tag-specifications  ResourceType=volume,Tags=[{Key=Name,Value=Task6EBS}]")  
        
        if ebs_vol == 0:
            print("EBS volume launched \n")
            os.system("chrome https://ap-south-1.console.aws.amazon.com/ec2/v2/home?region=ap-south-1#Volumes:sort=desc:createTime")
            time.sleep(300)
        else:
            print("There's something error here! EBS not deployed")
        
        
        print("=============================================")    
        print("Saving instance id and volume id in variables \n")
        instance_id = subprocess.getoutput("aws ec2 describe-instances --filters Name=tag:Name,Values=Task6 --query Reservations[*].Instances[*].[InstanceId] --output text")
        vol_id = subprocess.getoutput("aws ec2 describe-volumes  --filters Name=tag:Name,Values=Task6EBS --query Volumes[*].[VolumeId] --output text")
        
        print("=============================================")
        print("Attaching EBS volume")      
        ebs_att = os.system("aws ec2 attach-volume --instance-id {0} --volume-id {1} --device /dev/sdf".format(instance_id,vol_id))
        if ebs_att == 0:
            print("EBS volume attached with EC2 sucessfully \n")
            os.system("chrome https://ap-south-1.console.aws.amazon.com/ec2/v2/home?region=ap-south-1#InstanceDetails:instanceId=i-0529c713d9bd9e4d4")
            time.sleep(2)
            print("=============================================")
            print("Infrastructure deployed")
            print("============================================= \n")
            
        else:
            print("There's something error here! Can't attach ebs to ec2")
                 
    elif face_fake == 100:
        lock = cv2.imread("./image/lock.jpg")
        cv2.putText(lock, "Access", (45, 50) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(lock, "Denied", (45, 100) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow("lock" , lock)
        cv2.waitKey(4000)
        cv2.destroyAllWindows()
        
        # Detect face from image
        face_anon = face_classifier.detectMultiScale(image, 1.3, 5) 
        
        # Crop the image
        for x,y,w,h in face_anon:
            cimg1 = image[y:y+h, x:x+w]
            
                  
        # Below code was used to check output of cropped image
        #cv2.imshow("hii", cimg1)
        #cv2.waitKey(400)
        #cv2.destroyAllWindows()

        # Save the cropped image inside "image" folder
        cv2.imwrite("./image/Crop.jpg",cimg1)
        print("=============================================")

        print("We are sending mail to owner of this account \n")
        send_email()
        print("Mail Sent Successfully \n")
        print("=============================================")
        print("Sending whats app msg to owner")
        wpmsg()
        print("Message sent sucessfully to owner \n")

### Start Recognising your Face

In [15]:
# Load the Model 
deepak_model = cv2.face_LBPHFaceRecognizer.create() 
deepak_model.read("deepak_model.h5")

# Load haarcascade face classifier to detect face
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


def face_detector(img, size=0.5):
    
    #convert image to gray scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        #print("No image found")
        return img, []
    
    for (x,y,w,h) in faces:
        #print("faces found")
        cv2.rectangle(img, (x,w), (x+w, y+h), (0,255,255), 2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (500,500))
    return img, roi

# Open webcam
cap = cv2.VideoCapture(0)
face_actual = 0
face_fake = 0

while True:   
    ret, frame = cap.read()  
    image, face = face_detector(frame)
    #print("face detected")
        
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        # pass face to predict model
        # results consists a tuple having label & confidence value
        results = deepak_model.predict(face)
        #print(results)
        #print("face identified")
        
        if results[1] < 500:
            confidence = int(100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is user'
            
        
        cv2.putText(image, display_string, (50,100), cv2.FONT_HERSHEY_COMPLEX, 1, (80,120,0), 2)
        
        if confidence > 90:
            cv2.putText(image, "Hey Deepak!!", (220, 150), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow("Face Recognition", image)
            face_actual += 1
            
        else:
            cv2.putText(image, "I don't know, who are you", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Face Recognition", image)
            face_fake += 1
            
        if face_actual == 70:
            print("Actual Person Found")
            print("Your AWS infrastructure will be deployed") 
            break
            
        elif face_fake == 100:
            print("Fake Person Found")
            break
        
    except:
        cv2.putText(image, "No face found", (120, 200), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2) 
        cv2.imshow("Face Recognition", image)
        pass
        
    if cv2.waitKey(10) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()

aws_cli()
    

<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-15-f6666e3832d7>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Fake Person Found
We are sending mail to owner of this account 

Sending mail....
Email Sent sucessfully
Mail Sent Successfully 

Sending whats app msg to owner
Invalid time format
In 36 seconds web.whatsapp.com will open and after 10 seconds message will be delivered
Message sent sucessfully to owner 

